In [1]:

# Importation des librairies 

import pandas as pd 
import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import progressbar
import time
import sys
import os

    
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from imblearn.under_sampling import RandomUnderSampler 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import f1_score,classification_report,roc_auc_score,precision_score,recall_score, precision_recall_fscore_support 
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.preprocessing import StandardScaler
#from sklearn_som.som import SOM
import networkx as nx


pd.set_option('display.max_columns', 100)

from GEM.gem.utils      import graph_util, plot_util
from GEM.gem.evaluation import visualize_embedding as viz
from GEM.gem.evaluation import evaluate_graph_reconstruction as gr
from GEM.gem.embedding.gf       import GraphFactorization
#from GEM.gem.embedding.sdne     import SDNE
#from argparse import ArgumentParser
#from GraphEmbedding.ge import DeepWalk
#from GraphEmbedding.ge import SDNE
from karateclub.graph_embedding import Graph2Vec
from karateclub.node_embedding.neighbourhood import HOPE
from karateclub.node_embedding.neighbourhood import DeepWalk




num_frame=200#arbitraire , a tester plus serieusement

print("import terminé a :",time.strftime("%H:%M:%S", time.localtime()))

import terminé a : 10:22:59


In [2]:
def instals():
    !pip install progressbar2
    !pip3 install PyQt5
    
    

def create_graph(data_set):
    g = nx.MultiGraph()

    start = time.time()
    i=0
    while (i<len(data_set)):
        a=data_set["merchant"][i]
        b=data_set["cc_num"][i]
        g.add_edge(a,b,weight=1,edge_id=i)
        i=i+1
    
    print("---graph construction = %s seconds ---" % (time.time() - start));start = time.time()
    return g

def create_graph_deprecated(X_train_ultra_simple):
    g = nx.MultiGraph()

    start = time.time()
    i=0
    while (i<len(X_train_ultra_simple)):
        a=X_train_ultra_simple["merchant"][i]
        b=X_train_ultra_simple["cc_num"][i]
        g.add_edge(a,b,weight=0,edge_id=i)
        i=i+1
    i=0
    while (i<len(X_train_ultra_simple)):# on lis 2 fois mais ca coute que 4 sec
        a=X_train_ultra_simple["merchant"][i]
        b=X_train_ultra_simple["cc_num"][i]
        g[a][b]["weight"]=g[a][b]["weight"]+1 # il falais initialiser en premier
        i=i+1
    
    print("---graph construction = %s seconds ---" % (time.time() - start));start = time.time()
    return g

def fill(g,liste):
  
    while(liste):
        a=liste.pop()
        i=0
        while(i<len(liste)):
            b=liste[i]
            g.add_edge(a,b,weight=1)
            i+=1
    
    #return g

def init_sub_graph(nb_frames):
    # divison en plusieures sous graphs 
    sous_graph=[]
    i=0
    i2=0
    sub_g=0
    while ( i<nb_frames):
        sous_graph.append(nx.MultiGraph())
        i=i+1
    return sous_graph

def bipartite_dict(dict_merchants,dict_cc_num):

    dict_merchants_copy=dict_merchants.copy()
    dict_merchants_copy = dict([(value, key) for key, value in dict_merchants_copy.items()])
    dict_cc_num_copy=dict_cc_num.copy()
    dict_cc_num_copy = dict([(value, key) for key, value in dict_cc_num_copy.items()])

    for key in dict_merchants_copy.keys():
        dict_merchants_copy[key] = 0
    for key in dict_cc_num_copy.keys():
        dict_cc_num_copy[key] = 1
    return dict_merchants_copy,dict_cc_num_copy

def create_sub_graph(data_set,nb_frames,dict_merchants,dict_cc_num,start_at=0):
    sub_g=init_sub_graph(nb_frames)
    print (len(sub_g)," sous graphs")
    #sg in sub_g
    dict_merchants_copy,dict_cc_num_copy=bipartite_dict(dict_merchants,dict_cc_num)    
    time_frame_size=len(data_set) / nb_frames
    start = time.time()
    connected_count=0
    i=start_at
    data_set_size=len(data_set)+i
    i2=0
    sg=0
    while (i<data_set_size):
        a=data_set["merchant"][i]# rendre plus lisible
        b=data_set["cc_num"][i]
        sub_g[sg].add_edge(a,b,weight=1,edge_id=i,edge_rid=i2)

        i=i+1
        i2=i2+1
        if i2>= time_frame_size:
            nx.set_node_attributes(sub_g[sg], dict_merchants_copy, "bipartite")
            nx.set_node_attributes(sub_g[sg], dict_cc_num_copy, "bipartite")
            i2=0
            if(nx.is_connected(sub_g[sg])):
                connected_count+=1
            sg=sg+1

    print("---graph split = %s seconds ---" % (time.time() - start));start = time.time()

    return sub_g,connected_count


def create_sub_graph_deprecated(g,nb_frames,dict_merchants,dict_cc_num):
    sous_graph=init_sub_graph(nb_frames)
    dict_merchants_copy,dict_cc_num_copy=bipartite_dict(dict_merchants,dict_cc_num)    
    time_frame_size=len(X_train_ultra_simple) / num_frame
    start = time.time()
    connected_count=0
    i=0
    i2=0
    sub_g=0
    while (i<len(X_train_ultra_simple)):
        a=X_train_ultra_simple["merchant"][i]# rendre plus lisible
        b=X_train_ultra_simple["cc_num"][i]
        if(sous_graph[sub_g].has_edge(a,b)):
            sous_graph[sub_g][a][b]["weight"]=sous_graph[sub_g][a][b]["weight"]+1 
        else:
            sous_graph[sub_g].add_edge(a,b,weight=1,edge_id=i)
            #sous_graph[sub_g][a]["bipartite"]=0
            #sous_graph[sub_g][b]["bipartite"]=1
        i=i+1
        i2=i2+1
        if i2>= time_frame_size:
            nx.set_node_attributes(sous_graph[sub_g], dict_merchants_copy, "bipartite")
            nx.set_node_attributes(sous_graph[sub_g], dict_cc_num_copy, "bipartite")
            i2=0
            if(nx.is_connected(sous_graph[sub_g])):
                connected_count+=1
            sub_g=sub_g+1

    print("---graph split = %s seconds ---" % (time.time() - start));start = time.time()

    return sous_graph,connected_count


def invert_graph(gr):
    new_graph=nx.MultiGraph()
    #tous les arc d'un sommet sont connecté entre eux
    #step 1 = dans new_graph creer un sommet pour chaque arc

    for edge in gr.edges(data=True):
        #print(edge)
        node_id=edge[2]["edge_rid"]
        _edge_id=edge[2]["edge_id"]
        new_graph.add_node(node_id,edge_id=_edge_id)

    #pour chaque node de gr relier enssemble tous les arc
    for node in gr.nodes:
        _edge_list=list(gr.edges(node,data="edge_rid"))
        node_list=[]
        i=0
        #print(_edge_list)
        for items in _edge_list:
            #print(node[0])
            node_list.append(_edge_list[i][2])
            i=i+1            
        fill(new_graph,node_list)


    return new_graph

def invert_graph_deprecated(gr):
    new_graph=nx.MultiGraph()
    #tous les arc d'un sommet sont connecté entre eux
    #step 1 = dans new_graph creer un sommet pour chaque arc

    for edge in gr.edges(data=True):
        #print(edge)
        node_id=edge[2]["edge_rid"]
        new_graph.add_node(node_id)

        #print(" g[edge[0]][edge[1]][edge_id] = ",g[edge[0]][edge[1]]["edge_id"])
    #pour chaque node de g relier enssemble tous les arc
    for node in gr.nodes:
        dico=list(gr[node])# traitement de 1 node
        #print(dico)
        node_list=[]
        for key in dico:
            
            node_id=gr[node][key][0]["edge_rid"]# to test
            node_list.append(node_id)
        fill(new_graph,node_list)


    return new_graph


def create_inverted_sub_graph(sub_g):
    
    
    process_bar = progressbar.ProgressBar().start(max_value=len(sub_g))

    i=0
    start = time.time
    inv_sous_graph=[]
    for sg in sub_g:
        inv_sous_graph.append(invert_graph(sg))
        i=i+1
        process_bar.update(i)
        
    return inv_sous_graph
    print("---create_inverted_sub_graph  %s seconds ---" % (time.time() - start));start = time.time



def start_time_eval():
    start = time.time()
    i=0
    while(i<1000000):
        i=i+1
    boucle_time=time.time() - start
    start = time.time()
    i=0
    while(i<1000000):
        poubelle =time.time()
        i=i+1
    print("---1 milion de time.time=  %s seconds ---" % (time.time() - start-boucle_time));start = time.time()

    


#return le nombre d'arete du graph weighted_g
def nb_edge(weighted_g):
    summ=0
    NODES = list(weighted_g.nodes)
    for node in NODES:
        summ= summ+G.degree[node]
    return summ/2

def replissement(weighted_g, nb_merc,nb_cc_num):
    nb_edges=nb_edge(weighted_g)
    nb_max_edges =(nb_merc* nb_cc_num)#graph bipati
    return  nb_edges / nb_max_edges
    
def edge_repartition(g,len_dict_merchants):
    repartition=[]
    #remplissage de repartiotion avec 0 pour eviter les bugg
    
    #for each vertex 
        # for each edge in vertex.edges
            #repartition[ edge.poid ] ++
    print ("")
    

def slow_concat(d1,d2):
    return dict(d1.items() | d2.items())

def ditc_maping_so_slow_but_why(X_train_ultra_simple,dict_merchants,dict_cc_num):
    #---dictionary maping = 4272.313026428223 seconds ---
    start = time.time()
    size =len(X_train_ultra_simple)
    i=0
    time_val=[]
    while (i<size):
        X_train_ultra_simple.iat[i,0]=dict_merchants[X_train_ultra_simple.iat[i,0]]
        X_train_ultra_simple.iat[i,1]=dict_cc_num[X_train_ultra_simple.iat[i,1]]
        time_val.append(time.time() - start);start = time.time()
        i=i+1
    return time_val


def ditc_maping(X_train_ultra_simple,dict_merchants_cc_num):
    X_train_ultra_simple["merchant"].replace(dict_merchants_cc_num, inplace=True)
    X_train_ultra_simple["cc_num"].replace(dict_merchants_cc_num, inplace=True)

def ditc_maping_2(X_train_ultra_simple,dict_merchants_cc_num):
    X_train_ultra_simple["merchant"].replace(dict_merchants_cc_num, inplace=True)
    X_train_ultra_simple["cc_num"].replace(dict_merchants_cc_num, inplace=True)

    
def create_split_dict(X_train_ultra_simple,start_at=0):
    index=start_at
    start = time.time()
    dict_merchants=dict()
    dict_cc_num=dict()
    merc_id=0
    cc_id=0
    data_set_size= len(X_train_ultra_simple)+index
    while index < data_set_size:
        if X_train_ultra_simple["merchant"][index] not in dict_merchants.keys():
            dict_merchants[X_train_ultra_simple["merchant"][index]] = merc_id
            merc_id=merc_id+1
        index=index+1

    print("---remplissage dict_merchants  %s seconds ---" % (time.time() - start));start = time.time()
    index=start_at
    while index < data_set_size:
        if X_train_ultra_simple["cc_num"][index] not in dict_cc_num.keys():
            dict_cc_num[X_train_ultra_simple["cc_num"][index]] = merc_id
            merc_id=merc_id+1
        index=index+1

    print("---remplissage dict_cc_num %s seconds ---" % (time.time() - start));start = time.time()
    return dict_merchants,dict_cc_num

def create_dict(data_set):
    dict_merchants,dict_cc_num=create_split_dict(data_set)
    
    return slow_concat(dict_merchants,dict_cc_num)
#########################################################################
#########################################################################
#########################################################################
#########################################################################
#########################################################################
def create_dictionary_hell():# missing double values
    start = time.time()
    dictionary_hell=[]
    nodes=[]
    nb_sg=len(sous_graph)
    print(len(sous_graph))
    sg_index=0
    merc_id=0
    while sg_index < nb_sg:
        i=0
        merc_id=0
        nodes= list(sous_graph[sg_index].nodes)
        dictionary_hell.append(dict())
        while( i<len(nodes)):
            if nodes[i] not in dictionary_hell[sg_index].keys():
                dictionary_hell[sg_index][nodes[i]] = merc_id
                merc_id=merc_id+1
                
            i=i+1

        sg_index=sg_index+1
    print("--- dictionary__hell %s seconds ---" % (time.time() - start));start = time.time()
    return dictionary_hell

def relabel_graph(r_sous_graph,dictionary_hell,sous_graph):
    print ("relabel_",end="")
    start = time.time()
    i=0  
    while i<len(sous_graph):
        r_sous_graph.append(nx.relabel_nodes(sous_graph[i], dictionary_hell[i],copy=True))
        i=i+1
        print(".",end="")
    print("--- relabel_graph %s seconds ---" % (time.time() - start))

def unrelabel_graph(unr_sous_graph,dictionary_hell,sous_graph):
    start = time.time()
    i=0
    while i<len(dictionary_hell):
        if i==len(sous_graph):
            (print("errno=",i,end=','))
        inv_map = {v: k for k, v in dictionary_hell[i].items()}
        unr_sous_graph.append(nx.relabel_nodes(sous_graph[i], inv_map,copy=True))
        i=i+1
    print("--- revert labelling %s seconds ---" % (time.time() - start))


def print_info_diverses(X_train_ultra_simple,dico):
    print(X_train_ultra_simple["merchant"][0])
    print(X_train_ultra_simple["cc_num"][0])
    print(X_train_ultra_simple.columns)
    print(X_train_ultra_simple.loc[0]["merchant"])

    print (len(dico) , "humans in the system ")#1676
    print(X_train_ultra_simple["cc_num"][0])
    print (dict_merchants[ "fraud_Rippin, Kub and Mann"], type(dico[ "fraud_Rippin, Kub and Mann"]))
    

#laplacian similarity 1/2
def select_k(spectrum, minimum_energy = 0.9):#
    running_total = 0.0
    total = sum(spectrum)
    if total == 0.0:
        return len(spectrum)
    for i in range(len(spectrum)):
        running_total += spectrum[i]
        if running_total / total >= minimum_energy:
            return i + 1
    return len(spectrum)

#laplacian similarity 2/2
def laplacian_similarity(graph1,graph2):
    laplacian1 = nx.spectrum.laplacian_spectrum(graph1)
    laplacian2 = nx.spectrum.laplacian_spectrum(graph2)

    k1 = select_k(laplacian1)
    k2 = select_k(laplacian2)
    k = min(k1, k2)
    print("k selected =",k)
    similarity = sum((laplacian1[:k] - laplacian2[:k])**2)
    return similarity

def string_edit_dist():
    print("https://anhaidgroup.github.io/py_stringmatching/v0.3.x/Levenshtein.html")
def edit_dist_nx(g1,g2):
    
    for v in nx.optimize_graph_edit_distance(g1, g2):
        minv = v
    return minv


def graph_degree(g):
    degrees = [val for (node, val) in g.degree()]
    maxd=max(degrees)
    avg=sum(degrees)/len(degrees)
    #print("max degree =",maxd," mean = ",avg)
    return maxd,avg
def print_graph_info(connected_count):
    
    print(connected_count,"connected graphs")
    print(len(sous_graph[0].edges)," transactions ")
    print("sous_graph[0][0] , type = ",type(sous_graph[0][0]),"\n")
    print("sous_graph[0].nodes , type = ",type(sous_graph[0].nodes),"\n")
    print(list(sous_graph[0].nodes))
    print("------------")
    sub_g=0;node=0
    dico=dict(sous_graph[sub_g][node])
    for key in dico:
         print(sous_graph[0][0][key],"key = ",key)
    print(dict(sous_graph[0][0]))
    print("------------")
    print( type(sous_graph[0][0][693]))
    print(sous_graph[0][0][693])
    print("---------")
    print((sous_graph[0].edges))
    
def draw_1(g):
    start = time.time()
    #subax1 = plt.subplot()
    nx.draw(sous_graph[0], with_labels=False, node_size= 1)
    plt.savefig("draw_1.png")
    plt.show()
    print("---draw  %s seconds ---" % (time.time() - start));start = time.time
def draw_2(g):
    start = time.time()
    #subax2 = plt.subplot()
    options = {
        'node_size': 100,
        'width': 3,
    }
    nx.draw_spectral(g, **options)#approximation of the ratio cut
    plt.savefig("draw_2.png")
    plt.show()
    print("---draw  %s seconds ---" % (time.time() - start));start = time.time
def draw_3(g):
    start = time.time()
    #subax3 = plt.subplot()

    nx.draw_shell(g, with_labels=False,node_size= 1)# font_weight='bold')
    plt.savefig("draw_3.png")
    plt.show()
    print("---draw  %s seconds ---" % (time.time() - start));start = time.time
    
def draw_4(g,numb_merchant):
    total=len(g.nodes)#les valeures ascossié ne sont pas les bonnes mais c'est 
    X = list(range(0,numb_merchant ))# juste pour la position geographique 
    Y= list(range(numb_merchant,total ))
    pos = dict()
    pos.update( (n, (1, i)) for i, n in enumerate(X) ) # put nodes from X at x=1
    pos.update( (n, (2, i)) for i, n in enumerate(Y) ) # put nodes from Y at x=2
    nx.draw(g, pos=pos,node_size= 1)
    plt.savefig("draw_4.png")
    plt.show()

def drawing(g):
    %matplotlib inline
    draw_1(g)
    draw_2(g)
    draw_3(g)
    draw_4(g,len(dict_merchants.keys()))
    
    plt.show()
    
def complexity_calculations():
    val=0
    train_size=len(Y_train)
    test_size=len(Y_test)
    
    #l'idee premiere
    print("-----------------")
    print("tester la similarité de 1 graph modifié avec tt les autres ")
    print("pour 0.5 sec par calcul et ",num_frame," frames" )
    print( num_frame*0.5," sec")
    print("-----------------")
    print("modifier le graph et recomencer , pour chaques valeures dans train")
    print("pour completement calculer  les similarité de 1 transaction")
    print( num_frame*0.5*train_size," sec")
    print("donc ",int(num_frame*0.5*train_size/(3600*24))," jours")
    print("-----------------")
    
    big_number = num_frame*0.5*train_size*test_size
    big_number_year=int(big_number/31540000)
    print("pour un total de ",big_number," sec")
    print("donc ",big_number_year," ans")
    print("-----------------")
    print("en reduisant la precision au minimum")
    print("chaque transaction n'aura que 1 calcul de similarité")
    print("precision max 50% , doubler le temps de calcul double la precision")
    big_number=0.5*test_size
    print("pour un total de ",big_number," sec")
    print("donc ",int(big_number/3600)," heures")

    
def int_to_str(G):
    # convert nodes from int to str format
    keys = np.arange(0,int(len(dictionary.keys())))
    values = [str(i) for i in keys]
    dic = dict(zip(keys, values))
    H = nx.relabel_nodes(G, dic)

def try_catch_example():
    print('How many cats do you have?\n')
    numCats = input()
    try:
        if int(numCats) > 3:
            print('That is a lot of cats.')
        else:
            print('That is not that many cats.')
    except ValueError:
        print("Value error")  
def modif_graph_testing(graph):
    graph[1][2]["weight"]=2
    
def invert_node_testing():
    graph=nx.MultiGraph()
    graph.add_nodes_from([0,1,2,3,4])
    graph.add_edge(0,1,weight=0,edge_id="a")
    graph.add_edge(1,2,weight=0,edge_id="b")
    graph.add_edge(2,3,weight=0,edge_id="c")
    graph.add_edge(3,4,weight=0,edge_id="d")
    graph.add_edge(4,2,weight=0,edge_id="e")
    print(graph.nodes)
    print(graph.edges)
    print("avan 1--2--3--4")
    #new_graph=inver t_graph(graph)
    print("apres 1--2--3--4")
    print(new_graph.nodes)
    print(new_graph.edges)
    
    
    
def max_degree(inv_sg):
    _max=0
    for gr in inv_sg:
        current_max,mean =graph_degree(gr)
        if current_max>_max:
            _max=current_max
    print ("max degree= ",_max)
    return _max

def print_version():
    print ("python ",sys.version)
    import xgboost as xgb
    print("xgboost version = ",xgb.__version__)


def hope_on_inv_sg(inv_sg,dim=58,nb_frame=200):

    start = time.time()
    # hope un_attributed example
    model = HOPE(dimensions=dim)
    i=0
    model.fit(inv_sg[i])
    arr=model.get_embedding()
    i=1
    process_bar = progressbar.ProgressBar().start(max_value=nb_frame)
    while (i<nb_frame):
        # train the model and generate embeddings
        model.fit(inv_sg[i])
        arr=np.concatenate((arr,model.get_embedding()),axis=0)
        i+=1
        process_bar.update(i)
    print("---hope embedding = %s seconds ---" % (time.time() - start));start = time.time()
    return arr  

def hope_on_inv_sg_deprecated(inv_sg):

    start = time.time()
    # hope attributed example
    model = HOPE(dimensions=58)
    liste=[]
    i=0
    process_bar = progressbar.ProgressBar().start(max_value=num_frame)
    while (i<num_frame):
        # train the model and generate embeddings
        model.fit(inv_sg[i])
        if(liste==[]):
            emm=model.get_embedding()
            print(type(emm))
            liste.append(emm)
        #missing else ??????????????????????
        liste.extend(model.get_embedding())
        #ar=np.concatenate((ar,ar2),axis=0)
        i+=1
        process_bar.update(i)
    print("---hope embedding = %s seconds ---" % (time.time() - start));start = time.time()
    return liste  

def deepwalk_on_inv_sg(inv_sg,dim=58):


    start = time.time()
    # hope un_attributed example
    model = DeepWalk(dimensions=dim)
    i=0
    model.fit(inv_sg[i])
    arr=model.get_embedding()
    i=1
    process_bar = progressbar.ProgressBar().start(max_value=num_frame)
    while (i<num_frame):
        # train the model and generate embeddings
        model.fit(inv_sg[i])
        arr=np.concatenate((arr,model.get_embedding()),axis=0)
        i+=1
        process_bar.update(i)
    print("---deepwalk embedding = %s seconds ---" % (time.time() - start));start = time.time()
    return arr      

def deepwalk_on_inv_sg_deprecated(inv_sg):

    start = time.time()
    # hope attributed example
    model = DeepWalk(dimensions=58)
    liste=[]
    i=0
    process_bar = progressbar.ProgressBar().start(max_value=num_frame)
    while (i<num_frame):
        # train the model and generate embeddings
        
        model.fit(inv_sg[i])
        if(liste==[]):
            liste.append(model.get_embedding())
        liste.extend(model.get_embedding())
        i+=1
        process_bar.update(i)
    print("---deepwalk embedding = %s seconds ---" % (time.time() - start));start = time.time()
    return liste      



print("fonctions declaré a :",time.strftime("%H:%M:%S", time.localtime()))



fonctions declaré a : 10:22:59


In [3]:
##download data

program_start = time.time()

data_file= os.path.abspath('../../data')
full_path=data_file+'\\'+'fraudTrain.csv'
train_df=pd.read_csv(full_path)
full_path=data_file+'\\'+'fraudTest.csv'
test_df=pd.read_csv(full_path)

cols = train_df.columns.tolist()
cols = [c for c in cols if c not in ["is_fraud"]]
target = "is_fraud"
print(cols)

#Definition des nouvelles variables X_train and Y_train
X_train = train_df[cols]
Y_train = train_df[target]

#Definition des nouvelles variables X_test and Y_test
X_test = test_df[cols]
Y_test = test_df[target]

features = [ 'merchant', 'cc_num']
X_train = X_train[features]
X_test = X_test[features]

X_train_ultra_simple = X_train.copy()
X_train_ultra_simple=X_train_ultra_simple.iloc[:int(len(X_train_ultra_simple)*0.8)]
X_val_ultra_simple=X_train.copy().iloc[len(X_train_ultra_simple):]

X_test_ultra_simple = X_test.copy()
num_frame_test=round((len(X_test_ultra_simple)/len(X_train_ultra_simple))*num_frame)
num_frame_val=round((len(X_val_ultra_simple)/len(X_train_ultra_simple))*num_frame)

#you need to make a smaller number of subgraph
print(X_train_ultra_simple.shape)
print(X_val_ultra_simple.shape)
print(X_test_ultra_simple.shape)

['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat', 'merch_long']
(1037340, 2)
(259335, 2)
(555719, 2)


In [4]:

#remplissage des dictionaires
#_test

dict_merchants_train,dict_cc_num=create_split_dict(X_train_ultra_simple)
dict_merchants_val,dict_cc_num_val=create_split_dict(X_val_ultra_simple,start_at=len(X_train_ultra_simple))
dict_merchants_test,dict_cc_num_test=create_split_dict(X_test_ultra_simple)
dictionary=slow_concat(dict_merchants_train,dict_cc_num)
dictionary_val=slow_concat(dict_merchants_val,dict_cc_num_val)
dictionary_test=slow_concat(dict_merchants_test,dict_cc_num_test)

#dictionary=create_dict(X_train_ultra_simple)

# associer a chaque marchant son numero dans le dictionaire
#pour la lisibilité , et l'affichage
start = time.time()
ditc_maping(X_train_ultra_simple,dictionary)
ditc_maping(X_val_ultra_simple,dictionary_val)
ditc_maping(X_test_ultra_simple,dictionary_test)
print("---dictionary maping = %s seconds ---" % (time.time() - start));start = time.time()


---remplissage dict_merchants  5.637519598007202 seconds ---
---remplissage dict_cc_num 5.992034673690796 seconds ---
---remplissage dict_merchants  1.483436107635498 seconds ---
---remplissage dict_cc_num 1.53389310836792 seconds ---
---remplissage dict_merchants  3.07381272315979 seconds ---
---remplissage dict_cc_num 3.167191505432129 seconds ---
---dictionary maping = 72.46729111671448 seconds ---


In [5]:
print(X_val_ultra_simple["merchant"])

1037340      0
1037341      1
1037342      2
1037343      3
1037344      4
          ... 
1296670    564
1296671    246
1296672    646
1296673    121
1296674    353
Name: merchant, Length: 259335, dtype: int64


In [6]:


#pip install tk

    ##creation du graph
    
#g = create_graph(X_train_ultra_simple)#40 sec
#g_val=create_graph(X_val_ultra_simple)
#g_test= create_graph(X_test_ultra_simple)
#!jupyter notebook --generate-config
# divison en plusieures sous graphs #20 sec

sous_graph_train,connected_count=create_sub_graph(X_train_ultra_simple,num_frame,dict_merchants_train,dict_cc_num)
sous_graph_val,connected_count=create_sub_graph(X_val_ultra_simple,num_frame_val,dict_merchants_train,dict_cc_num,start_at=len(X_train_ultra_simple))
sous_graph_test,connected_count=create_sub_graph(X_test_ultra_simple,num_frame_test,dict_merchants_train,dict_cc_num)

#dictionary_hell=create_dictionary_hell()
#pour afficher , attention au cascades
#!jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

print(connected_count," connected graphs")

print("---depuis le debut  %s seconds ---" % (time.time() - program_start));start = time.time
#!git clone https://github.com/shenweichen/GraphEmbedding.git


if(False):
    liste=[]
    i=0
    num_frame=len(sous_graph)
    G = sous_graph[i]
    liste.append(encapsulation(G))
    i+=1

    G = sous_graph[i]
    liste.append(encapsulation(G))
    i+=1

#crash_free()
#!pip3 install PyQt5
# affichage

#drawing(sous_graph[0])


#370 MiB = 300 000 Mo
# avec g/200 ca compile , tres .... lentement , mais ca compile
too_long=True
if(not too_long):
    val = edit_dist_nx(sous_graph[0],sous_graph[1])

affichage=False
if(affichage):
    complexity_calculations()
    drawing(g)
    print_graph_info(connected_count)


200  sous graphs
---graph split = 17.942409992218018 seconds ---
50  sous graphs
---graph split = 4.491936206817627 seconds ---
107  sous graphs
---graph split = 9.022253513336182 seconds ---
85  connected graphs
---depuis le debut  132.8589255809784 seconds ---


In [7]:
debugging=False
if (debugging):
    sous_graph,connected_count=create_sub_graph(X_train_ultra_simple,num_frame,dict_merchants,dict_cc_num)
    sous_graph_test,connected_count=create_sub_graph(X_test_ultra_simple,num_frame_test,dict_merchants,dict_cc_num)


In [8]:
print(num_frame)
print(len(sous_graph_train))
print(len(sous_graph_test))
print(num_frame_test)


200
200
107
107


In [9]:
### partie karateclub
########################################################################
########################################################################
########################################################################
########################################################################
########################################################################
########################################################################
########################################################################
########################################################################
########################################################################
########################################################################

In [10]:
#graph construction

start = time.time()


inv_sg_train= create_inverted_sub_graph(sous_graph_train)
inv_sg_val = create_inverted_sub_graph(sous_graph_val)
inv_sg_test= create_inverted_sub_graph(sous_graph_test)
print("--- graph inversion = %s seconds ---" % int(time.time() - start));start = time.time()


100% (107 of 107) |######################| Elapsed Time: 0:00:22 ETA:  00:00:00

--- graph inversion = 69 seconds ---


In [11]:

def is_it_missing_a_number(whatever):
    missing=False
    if isinstance(whatever,dict):
    
        l=sorted(whatever.keys())
        i=0
        while i<len(l):
            if l[i]!=i:
                missing=True
            i=i+1
        i=i-1
        #print("len(dict)= ",len(l)," , lastval = ",l[i],end=" ")
        if(missing):
            print(" missing a number",end=" ")
            return True
    if isinstance(whatever,list):
        l=sorted(whatever)
        i=0
        while i<len(l):
            if l[i]!=i:
                missing=True
            i=i+1
        i=i-1
        #print("len(list)= ",len(l)," , lastval = ",l[i],end=" ")
        if(missing):
            print(" missing a number",end=" ")
            return True
    #print("\n")
    return False
    
def is_inv_working():
    print (len(inv_sg[30].nodes))
    l1=list(inv_sg[30].nodes)
    l2=list(unr_sous_graph[30].nodes)
    i=0
    while(i<len( inv_sg[30])):
        if(l1[i] != l2[i]):
            print("not_the_same")
        i=i+1

    i=0# il y a au moins les memes values
    for k in inv_sg[30] :
        #print(",",end="");i=i+1
        if k not in(unr_sous_graph[30] ):
            print("not_the_same")
    print(i)
    
def is_inv_conservating_order(inv_sg):
    node_list=[]
    for node in inv_sg.nodes(data=True):
        
        node_list.append([node[0],node[1]["edge_id"]])
        #[inv_sg.nodes(node,data="node_id"),inv_sg.nodes(node,data="edge_id")]
    #print (np.array(node_list))
    return (np.array(node_list))

In [12]:
#node_list=is_inv_conservating_order(inv_sg_train[10])
#node_list.sort(axis=0)
#print(node_list)

md=max_degree(inv_sg_train)
md2=max_degree(inv_sg_val)
md3=max_degree(inv_sg_test)
print (len(inv_sg_train))
print (len(inv_sg_test))#should be 107
print("len inv_sg = ",len(inv_sg_train),len(inv_sg_train)*len(inv_sg_train[0])," / len xtrain = ",len(X_train_ultra_simple))
print("len inv_sg_val = ",len(inv_sg_val)," / len xval = ",len(X_val_ultra_simple))
print("len inv_sg_test = ",len(inv_sg_test)," / len xtest = ",len(X_test_ultra_simple))

max degree=  48
max degree=  48
max degree=  47
200
107
len inv_sg =  200 1037400  / len xtrain =  1037340
len inv_sg_val =  50  / len xval =  259335
len inv_sg_test =  107  / len xtest =  555719


In [13]:
#print(sorted(list(r_sous_graph[0].nodes)))
#print(sorted((dictionary_hell[0]).keys()))
#is_it_missing_a_number(dictionary_hell[1])
i=0
while(i<len(inv_sg_train)):
    nodes = inv_sg_train[i].nodes(data=False)
    is_it_missing_a_number(list(nodes))
    i=i+1
# Graph2Vec attributed example
if(False):
    small_sous_graph=[r_sous_graph[0],r_sous_graph[1]]
    
    model = Graph2Vec(attributed=False)
    model.fit(small_sous_graph)
    ecmb=model.get_embedding()

    print(type(range(r_sous_graph[0].number_of_nodes())))

    print("---embedding = %s seconds ---" % (time.time() - start));start = time.time()


In [14]:
#---hope embedding = 354.7910692691803 seconds ---
md=max_degree(inv_sg_train)
emb_train = hope_on_inv_sg(inv_sg_train,dim=md,nb_frame=num_frame)
emb_val = hope_on_inv_sg(inv_sg_val,dim=md,nb_frame=num_frame_val)

emb_test = hope_on_inv_sg(inv_sg_test,dim=md,nb_frame=num_frame_test)


max degree=  48


100% (200 of 200) |######################| Elapsed Time: 0:02:04 ETA:  00:00:00

---hope embedding = 124.71520638465881 seconds ---


100% (50 of 50) |########################| Elapsed Time: 0:00:36 ETA:  00:00:00

---hope embedding = 37.264121770858765 seconds ---


100% (107 of 107) |######################| Elapsed Time: 0:01:05 ETA:  00:00:00

---hope embedding = 66.31422781944275 seconds ---


In [15]:
#---deepwalk embedding = 6973.2731170654297 seconds ---
#deepwalk_emb=deepwalk_on_inv_sg(inv_sg)
#deepwalk_emb=deepwalk_on_inv_sg(inv_sg,max_degree(inv_sg))# variable size

In [16]:
# ------------------- fin de la partie graph
################################################################
################################################################
################################################################
################################################################
################################################################
################################################################
################################################################
################################################################
################################################################
################################################################
################################################################
#-----------------   debut de la partie IA
#  xgboost lstm random_forest neural_network
# svm ---> +70 heure d'exe

In [17]:
#import le data_set complet pour xgboost et cie

full_path=data_file+'\\'+'X_train_1_2_svm.csv'
xtrain_transformed_complique=pd.read_csv(full_path)
ytrain_transformed_complique=train_df['is_fraud'].iloc[:int(len(train_df)*0.8)]

full_path=data_file+'\\'+'X_val_1_2_svm.csv'
xval_transformed_complique=pd.read_csv(full_path)
yval_transformed_complique=train_df['is_fraud'].iloc[len(ytrain_transformed_complique):]


full_path=data_file+'\\'+'X_test_1_2_svm.csv'
xtest_transformed_complique=pd.read_csv(full_path)
ytest_transformed_complique=test_df['is_fraud']




train_df=train_df.drop(columns=['Unnamed: 0'])
test_df=test_df.drop(columns=['Unnamed: 0'])
xtrain_transformed_complique=xtrain_transformed_complique.drop(columns=['Unnamed: 0'])
xval_transformed_complique=xval_transformed_complique.drop(columns=['Unnamed: 0'])
xtest_transformed_complique=xtest_transformed_complique.drop(columns=['Unnamed: 0'])

cols = xtrain_transformed_complique.columns.tolist()
print(cols)


['merchant', 'category', 'amt', 'gender', 'state', 'zip', 'lat', 'long', 'city_pop', 'dob', 'unix_time', 'merch_lat', 'merch_long', 'delta_time', 'delta_amt', 'delta_time_category', 'delta_amt_category', 'delta_time_merchant', 'delta_amt_merchant', 'avg_amt', 'delta_avg_amt', 'avg_amt_category', 'delta_avg_amt_category', 'avg_amt_merchant', 'avg_amt_state', 'avg_amt_city', 'avg_amt_job', 'delta_avg_amt_category_job', 'month', 'day', 'hour']


In [18]:
#add columns
print((emb_train.shape))
print(xtrain_transformed_complique.shape)
emb_xtrain_concat=np.concatenate((xtrain_transformed_complique,emb_train),axis=1)
emb_xval_concat=np.concatenate((xval_transformed_complique,emb_val),axis=1)
emb_xtest_concat=np.concatenate((xtest_transformed_complique,emb_test),axis=1)

print(emb_xtrain_concat.shape)

(1037340, 48)
(1037340, 31)
(1037340, 79)


In [19]:
# test avec xgboost
from xgboost import XGBClassifier

In [20]:
# sans modification
start = time.time()

xgb = XGBClassifier(n_estimators=50, gamma=0.05,eta=0.05,max_depth=7, n_jobs=16)

xgb.fit(xtrain_transformed_complique,ytrain_transformed_complique)
Y_train_pred=xgb.predict(xtrain_transformed_complique)
Y_test_pred=xgb.predict(xtest_transformed_complique)
print(classification_report(ytrain_transformed_complique,Y_train_pred))
print(classification_report(Y_test,Y_test_pred))

print("---XGboost sans modification = %s seconds ---" % (time.time() - start));start = time.time()


C:\Users\benjamin.marty\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:30:35] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1031372
           1       0.98      0.83      0.90      5968

    accuracy                           1.00   1037340
   macro avg       0.99      0.91      0.95   1037340
weighted avg       1.00      1.00      1.00   1037340

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.95      0.79      0.86      2145

    accuracy                           1.00    555719
   macro avg       0.97      0.89      0.93    555719
weighted avg       1.00      1.00      1.00    555719

---XGboost sans modification = 81.52752661705017 seconds ---


In [21]:
#ajout du graph

xgb.fit(emb_xtrain_concat,ytrain_transformed_complique)
Y_train_pred_emb=xgb.predict(emb_xtrain_concat)
Y_test_pred_emb=xgb.predict(emb_xtest_concat)
print(classification_report(ytrain_transformed_complique,Y_train_pred_emb))
print(classification_report(Y_test,Y_test_pred_emb))

print("---XGboost avec modification = %s seconds ---" % (time.time() - start));start = time.time()


C:\Users\benjamin.marty\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:32:04] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1031372
           1       0.98      0.83      0.90      5968

    accuracy                           1.00   1037340
   macro avg       0.99      0.91      0.95   1037340
weighted avg       1.00      1.00      1.00   1037340

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.95      0.79      0.86      2145

    accuracy                           1.00    555719
   macro avg       0.97      0.89      0.93    555719
weighted avg       1.00      1.00      1.00    555719

---XGboost avec modification = 300.9893820285797 seconds ---


In [22]:
if(False):# surement plus de 4 h
    import shap
    start = time.time()
    explainer = shap.TreeExplainer(xgb) 
    shap_values = explainer.shap_values(emb_xtest_concat) 
    shap.summary_plot(shap_values, emb_xtest_concat, plot_type="bar")
    print("---shap for XGboost = %s seconds ---" % (time.time() - start));start = time.time()

In [23]:
print("xgboost")
cl=classification_report(Y_test,Y_test_pred,output_dict=True)
cl2=classification_report(Y_test,Y_test_pred_emb,output_dict=True)
print("recall diff = %.4f" % (cl2["1"]["recall"]-cl["1"]["recall"]))
print("f1-score diff = %.4f" % (cl2["1"]["f1-score"]-cl["1"]["f1-score"]))


recall diff = -0.0005
f1-score diff = -0.0005


In [24]:
print(xtrain_transformed_complique.columns)
print(Y_train_pred.shape)

Index(['merchant', 'category', 'amt', 'gender', 'state', 'zip', 'lat', 'long',
       'city_pop', 'dob', 'unix_time', 'merch_lat', 'merch_long', 'delta_time',
       'delta_amt', 'delta_time_category', 'delta_amt_category',
       'delta_time_merchant', 'delta_amt_merchant', 'avg_amt', 'delta_avg_amt',
       'avg_amt_category', 'delta_avg_amt_category', 'avg_amt_merchant',
       'avg_amt_state', 'avg_amt_city', 'avg_amt_job',
       'delta_avg_amt_category_job', 'month', 'day', 'hour'],
      dtype='object')
(1037340,)


In [25]:
#lstm
#print((xtrain_transformed_complique_lstm.shape))
#print((xtrain_transformed_complique_lstm.shape[1:]))
#print(ytrain_transformed_complique.shape)

In [26]:
it_works="it_definitly_doesnt_work"
if(it_works=="yes_it_does"):
    from tensorflow.keras.layers import LSTM, Layer, Dense
    from tensorflow.keras.layers import Masking
    from tensorflow.keras.models import Sequential
    import tensorflow as tf
    from tensorflow.keras import metrics
    BATCH_SIZE = 32

    xtrain_transformed_complique_lstm=np.expand_dims(xtrain_transformed_complique, 1) 
    xval_transformed_complique_lstm=np.expand_dims(xval_transformed_complique, 1) 
    xtest_transformed_complique_lstm=np.expand_dims(xtest_transformed_complique, 1) 


    lstm = Sequential()
    lstm.add(LSTM(50, input_shape=(xtrain_transformed_complique_lstm.shape[1:]), return_sequences = True))
    #lstm.add(LSTM(50, input_shape=xtrain_transformed_complique.shape[1:], return_sequences = True))

    lstm.add(Dense(1,activation='sigmoid'))
    lstm.summary()
    #compile and fit the model
    earlystopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=3, mode='auto', baseline=None, restore_best_weights=True)
    lstm.compile(optimizer = 'adam',loss = 'binary_crossentropy' ,metrics = ['accuracy', metrics.Precision(class_id=1), metrics.Recall(class_id=1)])


    history = lstm.fit(xtrain_transformed_complique_lstm, ytrain_transformed_complique, verbose=1, validation_data = (xval_transformed_complique_lstm, yval_transformed_complique), epochs = 50, batch_size = BATCH_SIZE, shuffle = False, callbacks = [earlystopper])
    Y_train_pred = lstm.predict_classes(xtrain_transformed_complique_lstm, batch_size=BATCH_SIZE)
    Y_test_pred = lstm.predict_classes(xtest_transformed_complique_lstm, batch_size=BATCH_SIZE)

    print(classification_report(ytrain_transformed_complique,Y_train_pred))
    print(classification_report(Y_test,Y_test_pred))

    print("---XGboost avec modification = %s seconds ---" % (time.time() - start));start = time.time()



In [27]:
#Neural Network

In [30]:
# manual f1 score 
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

#trouve le meilleur dict
def find_best_dict(dict_list):
    threshold=0.92
    while(threshold>0.50):
        tmp_list=list()
        for di in dict_list:
            precision=(di["1"]["precision"])
            if(precision>threshold):
                tmp_list.append(di)
        if(tmp_list):
            newlist = sorted(tmp_list, key=lambda d: d["1"]['f1-score'],reverse=True)
            print("choosen model precision: %.4f" %newlist[0]["1"]["precision"]," f1-score: %.4f" %newlist[0]["1"]['f1-score'])
            return newlist[0]
        threshold=threshold-0.04
    print("nothing good was found")
    return_value=sorted(dict_list, key=lambda d: d["1"]['f1-score'],reverse=True)
    return return_value[0]


def fixed_nn_exec_time(xtrain_transformed_complique,ytrain_transformed_complique ,
                        xtest_transformed_complique,ytest_transformed_complique ,
                        tmps_exec=30,max_iter=1000):
    start_function = time.time()
    results=[]
    curent_iter=0
    while((tmps_exec*60)>(time.time() - start_function)):
        model = Sequential()
        model.add(Dense(32, input_shape=(len(xtrain_transformed_complique.columns),),
                        activation='relu')),
        model.add(BatchNormalization()),
        model.add(Dense(16, activation='relu')),
        model.add(Dense(8, activation='relu')),
        model.add(Dropout(0.2)),
        model.add(Dense(4, activation='relu')),
        model.add(Dense(1, activation='sigmoid'))
        opt = tf.keras.optimizers.Adam(learning_rate=0.001) #optimizer
        los=tf.keras.losses.BinaryCrossentropy()
        model.compile(optimizer=opt, loss="binary_crossentropy", 
                      metrics=[tf.keras.metrics.Precision(),"accuracy",f1])

        history= model.fit(xtrain_transformed_complique, ytrain_transformed_complique
                           ,epochs = 10, batch_size=128, verbose = 0)
        history_dictict = history.history
        y_test_pred =(model.predict(xtest_transformed_complique) >0.5).astype("int32")
        print(classification_report(ytest_transformed_complique,y_test_pred))

        cl_nn=classification_report(ytest_transformed_complique,y_test_pred
                                    ,output_dict=True)
        results.append(cl_nn)

        curent_iter=curent_iter+1
        if curent_iter > max_iter:
            find_best_dict(results)
        
    return find_best_dict(results)
    

In [31]:
nn_results=[]
print("start nn no embeding") 
nn_res=fixed_nn_exec_time(xtrain_transformed_complique,ytrain_transformed_complique ,
                        xtest_transformed_complique,ytest_transformed_complique ,
                        tmps_exec=30,max_iter=1000)
nn_results.append(nn_res)
print("---neural network = %s seconds ---" % (time.time() - start));start = time.time()



df_addon = pd.DataFrame(emb_train)
X_train_plus_graph_df=pd.concat([xtrain_transformed_complique,df_addon],axis=1)

df_addon = pd.DataFrame(emb_val)
X_val_plus_graph_df=pd.concat([xval_transformed_complique,df_addon],axis=1)

df_addon = pd.DataFrame(emb_test)
X_test_plus_graph_df=pd.concat([xtest_transformed_complique,df_addon],axis=1)


print("start nn with embeding") 
nn_res=fixed_nn_exec_time(X_train_plus_graph_df,ytrain_transformed_complique ,
                    X_test_plus_graph_df,ytest_transformed_complique ,
                    tmps_exec=30)
nn_results.append(nn_res)
print("---neural network = %s seconds ---" % (time.time() - start));start = time.time()



start nn no embeding
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.86      0.84      0.85      2145

    accuracy                           1.00    555719
   macro avg       0.93      0.92      0.93    555719
weighted avg       1.00      1.00      1.00    555719

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.90      0.78      0.84      2145

    accuracy                           1.00    555719
   macro avg       0.95      0.89      0.92    555719
weighted avg       1.00      1.00      1.00    555719

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.77      0.84      0.81      2145

    accuracy                           1.00    555719
   macro avg       0.89      0.92      0.90    555719
weighted avg       1.00      1.00      1.00    555719

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.89      0.75      0.81      2145

    accuracy                           1.00    555719
   macro avg       0.94      0.87      0.91    555719
weighted avg       1.00      1.00      1.00    555719

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.89      0.75      0.81      2145

    accuracy                           1.00    555719
   macro avg       0.94      0.87      0.91    555719
weighted avg       1.00      1.00      1.00    555719

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.84      0.79      0.82      2145

    accuracy                           1.00    555719
   macro avg       0.92      0.90      0.91    555719
weighted avg       1.00      1.00      1.00    555719

              preci

In [32]:
cl=nn_results[0]
cl2=nn_results[1]
print("nn recall diff = %.4f" % (cl2["1"]["recall"]-cl["1"]["recall"]))
print("nn f1-score diff = %.4f" % (cl2["1"]["f1-score"]-cl["1"]["f1-score"]))


nn recall diff = -0.0224
nn f1-score diff = -0.0142


In [33]:
start = time.time()
from sklearn.ensemble import RandomForestClassifier

rf_results=[]


max_depth = 15
rf = RandomForestClassifier(random_state=0, max_depth=max_depth, min_samples_leaf= 3, min_samples_split= 2, n_estimators= 140, verbose=1, n_jobs=16)
rf.fit(xtrain_transformed_complique, ytrain_transformed_complique)
# Prediction for the training/validation set
y_train_pred = rf.predict(xtrain_transformed_complique)
y_test_pred = rf.predict(xtest_transformed_complique)
rf_res=classification_report(ytest_transformed_complique,y_test_pred
                            ,output_dict=True)
rf_results.append(rf_res)
print("---RandomForest sans embedding = %s seconds ---" % (time.time() - start));start = time.time()

max_depth = 15
rf = RandomForestClassifier(random_state=0, max_depth=max_depth, min_samples_leaf= 3, min_samples_split= 2, n_estimators= 140, verbose=1, n_jobs=16)
rf.fit(emb_xtrain_concat, ytrain_transformed_complique)
# Prediction for the training/validation set
y_train_pred_emb = rf.predict(emb_xtrain_concat)
y_test_pred_emb = rf.predict(emb_xtest_concat)
rf_res=classification_report(ytest_transformed_complique,y_test_pred_emb
                            ,output_dict=True)
rf_results.append(rf_res)
print("---RandomForest avec embedding = %s seconds ---" % (time.time() - start));start = time.time()


[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:   51.2s
[Parallel(n_jobs=16)]: Done 140 out of 140 | elapsed:  4.1min finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.8s
[Parallel(n_jobs=16)]: Done 140 out of 140 | elapsed:    4.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.3s
[Parallel(n_jobs=16)]: Done 140 out of 140 | elapsed:    2.0s finished


---RandomForest sans embedding = 249.9911766052246 seconds ---


[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:  1.7min
[Parallel(n_jobs=16)]: Done 140 out of 140 | elapsed:  7.8min finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.7s
[Parallel(n_jobs=16)]: Done 140 out of 140 | elapsed:    4.1s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.3s


---RandomForest avec embedding = 479.1933732032776 seconds ---


[Parallel(n_jobs=16)]: Done 140 out of 140 | elapsed:    2.2s finished


In [34]:
print(classification_report(ytrain_transformed_complique,y_train_pred))
print(classification_report(ytest_transformed_complique,y_test_pred))
print("-------------------------------------------------------------------")
print(classification_report(ytrain_transformed_complique,y_train_pred_emb))
print(classification_report(ytest_transformed_complique,y_test_pred_emb))
print("-------------------------------------------------------------------")
cl=rf_results[0]
cl2=rf_results[1]
print("rf recall diff = %.4f" % (cl2["1"]["recall"]-cl["1"]["recall"]))
print("rf f1-score diff = %.4f" % (cl2["1"]["f1-score"]-cl["1"]["f1-score"]))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1031372
           1       1.00      0.83      0.91      5968

    accuracy                           1.00   1037340
   macro avg       1.00      0.92      0.95   1037340
weighted avg       1.00      1.00      1.00   1037340

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.97      0.73      0.83      2145

    accuracy                           1.00    555719
   macro avg       0.99      0.87      0.92    555719
weighted avg       1.00      1.00      1.00    555719

-------------------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1031372
           1       1.00      0.77      0.87      5968

    accuracy                           1.00   1037340
   macro avg       1.00      0.88      0.93   1037340
weight

In [36]:
svm_results=[]

In [35]:
from sklearn import svm

start = time.time()
svm_model = svm.SVC(kernel="rbf", gamma = 0.02, C=10)#, max_iter=100000)
svm_model.fit(xtrain_transformed_complique,ytrain_transformed_complique)
print("--- svm fit = %s seconds ---" % (time.time() - start));start = time.time()
y_test_pred=svm_model.predict(xtest_transformed_complique)
print("---SVM y_test_pred = %s seconds ---" % (time.time() - start));start = time.time()

print(classification_report(ytest_transformed_complique,y_test_pred))


cl_svm=classification_report(ytest_transformed_complique,y_test_pred
                            ,output_dict=True)
svm_results.append(cl_svm)
print("--------------------------------------------------------")

--- svm fit = 2504.10399889946 seconds ---
---SVM y_test_pred = 521.0672700405121 seconds ---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.91      0.66      0.76      2145

    accuracy                           1.00    555719
   macro avg       0.95      0.83      0.88    555719
weighted avg       1.00      1.00      1.00    555719



NameError: name 'svm_results' is not defined

In [38]:
print("svm start")

start_svm = time.time()
svm_model = svm.SVC(kernel="rbf", gamma = 0.02, C=10)#, max_iter=100000)
svm_model.fit(emb_xtrain_concat,ytrain_transformed_complique)
print("--- svm fit = %s seconds ---" % (time.time() - start));start = time.time()
Y_test_pred_graph=svm_model.predict(emb_xtest_concat)
print("---SVM y_test_pred = %s seconds ---" % (time.time() - start));start = time.time()

print(classification_report(ytest_transformed_complique,Y_test_pred_graph))
svm_results.append(classification_report(ytest_transformed_complique,Y_test_pred_graph,output_dict=True))

print("---svm = %s seconds ---" % (time.time() - start_svm));start = time.time()


svm start
--- svm fit = 7599.587657213211 seconds ---
---SVM y_test_pred = 865.5228717327118 seconds ---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.91      0.63      0.75      2145

    accuracy                           1.00    555719
   macro avg       0.96      0.82      0.87    555719
weighted avg       1.00      1.00      1.00    555719

---svm = 6004.418404579163 seconds ---


In [39]:
cl=svm_results[0]
cl2=svm_results[1]
print("nn recall diff = %.4f" % (cl2["1"]["recall"]-cl["1"]["recall"]))
print("nn f1-score diff = %.4f" % (cl2["1"]["f1-score"]-cl["1"]["f1-score"]))


nn recall diff = -0.0242
nn f1-score diff = -0.0152


In [40]:
print_version()

python  3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]
xgboost version =  1.3.3


In [42]:
#run clustering techniques on the embeded trash
